# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -7.916456464963                   -0.69    4.8    150ms
  2   -7.921207972097       -2.32       -1.52    1.0   53.1ms
  3   -7.921405430864       -3.70       -2.49    1.4   57.4ms
  4   -7.921441012098       -4.45       -2.83    2.6   74.2ms
  5   -7.921441646071       -6.20       -3.02    1.1   54.7ms
  6   -7.921442007749       -6.44       -4.55    1.0   54.6ms
  7   -7.921442021938       -7.85       -4.72    2.9    108ms
  8   -7.921442022133       -9.71       -5.39    1.0   53.5ms
  9   -7.921442022137      -11.34       -5.23    1.8   61.8ms
 10   -7.921442022144      -11.19       -6.01    1.0   54.5ms
 11   -7.921442022144      -12.48       -6.95    1.5   59.4ms
 12   -7.921442022144      -13.53       -7.12    2.5   74.1ms
 13   -7.921442022144      -14.57       -7.64    1.0   54.2ms
 14   -7.921442022144   +    -Inf       -7.88    1.0   60.2ms
 15   -7

Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   1.97s  Non-interacting
   1        0       1        0.31     52.4   9.15s  
   2        0       2       -0.52     46.9   1.35s  
   3        0       3       -2.40     41.7   1.16s  
   4        0       4       -4.46     29.6   916ms  
   5        0       5       -6.46     17.4   674ms  
   6        0       6       -8.91      4.0   404ms  
   7        1       1       -7.83     55.5   1.89s  Restart
   8        1       2       -8.92      5.6   453ms  
                                      72.8   1.70s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      72.9   1.52s  Non-interacting
   1        0       1        0.31     52.4   1.47s  
   2        0       2       -0.52     46.9   2.03s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.00349815535508104
 -0.002977451155014544
 -0.002977451154421335
  0.004302230630015751
  0.004302230630401164
  0.0043532017614127845